<div class="alert alert-block alert-info">
<b>Deadline:</b> March 22, 2023 (Wednesday) 23:00
</div>

# Exercise 2. Recommender system

In this exercise, your task is to design a recommender system.

## Learning goals:
* Practise tuning a neural network model by using different regularization methods.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [29]:
skip_training = True  # Set this flag to True before validation and submission

In [3]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the movie (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5.

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [7]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [8]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

You need to design a recommender system model with the API described in the cell below.

Hints on the model architecture:
* You need to use [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding) layer to convert inputs `user_ids` and `item_ids` into reasonable representations. The idea of the embedding layer is that we want to represent similar users with values that are close to each other. The original representation as integers is not good for that. By using the embedding layer, we can learn such useful representations automatically.

### Model tuning

In this exercise, you need to tune the architecture of your model to achieve the best performance on the provided test set. You will notice that overfitting is a severe problem for this data: The model can easily overfit the training set producing poor accuracy on the out-of-training (test) data.

You need to find an optimal combination of the hyperparameters, with some hyperparameters corresponding to the regularization techniques that we studied in the lecture.

The hyperparameters that you are advised to consider:
* Learning rate value and learning rate schedule (decresing the learning rate often has positive effect on the model performance)
* Number of training epochs
* Network size
* Weight decay
* Early stopping
* Dropout
* Increase amount of data:
  * Data augmentation
  * Injecting noise

You can tune the hyperparameters by, for example, grid search, random search or manual tuning. In that case, you can use `architecture` argument to specify the hyperparameters that define the architecture of your network. After you have tuned the hyperparameters, set the default value of this argument to the optimal set of the hyparameters so that the best architecture is used in the accuracy tests.

Note:
* The number of points that you will get from this exercise depends on the MSE loss on the test set:
  * below 1.00: 1 point
  * below 0.95: 2 points
  * below 0.92: 3 points
  * below 0.90: 4 points
  * below 0.89: 5 points
  * below 0.88: 6 points 

In [9]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items,
                 architecture=None  # If you want to tune the hyperparameters automatically (e.g. using random
                                    # search), use this argument to specify the hyperparameters that define the
                                    # architecture of your network. After you have tuned the hyperparameters,
                                    # set the default value of this argument to the optimal set of the hyparameters
                                    # so that the best architecture is used in the accuracy tests.
                ):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        # YOUR CODE HERE
        super().__init__()
        
        # user and item embedding layers
        
        self.user_embedding = nn.Embedding(num_embeddings=n_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=n_items, embedding_dim=8)
        
        # user and item embedding layers
        
        self.fc1 = nn.Linear(in_features=16, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=16)
        self.output = nn.Linear(in_features=16, out_features=1)
        #self.dropout = nn.Dropout(0.20)
        
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        # YOUR CODE HERE
    
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_ids-1)
        item_embedded = self.item_embedding(item_ids-1)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = torch.tanh(self.fc1(vector))
        #vector = self.dropout(vector)
        vector = torch.tanh(self.fc2(vector))
        #vector = self.dropout(vector)

        # Output layer
        pred = self.output(vector)

        return pred.flatten()
    

You can test the shapes of the model outputs using the function below.

In [10]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [11]:
# This cell is reserved for testing

## Train the model

You need to train a recommender system using **only the training data.** Please use the test set to select the best model: the model that generalizes best to out-of-training data.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models.

In [12]:
# Create the model
# IMPORTANT: the default value of the architecture argument should define your best model.
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [21]:
# Implement the training loop in this cell

# This function computes the accuracy on the test dataset
#def compute_accuracy(model, testloader):
    #model.eval()
    #correct = 0
    #total = 0
    #with torch.no_grad():
        #for user_id, itm_id, rating in testloader:
            #user_id, itm_id, rating = user_id.to(device), itm_id.to(device), rating.to(device)
            #outputs = model(user_id, itm_id)
            #total += rating.size(0)
            #correct += (torch.round(outputs) == rating).sum().item()
            #return correct / total
            
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)
        

# This is the function to compute the loss:
def compute_loss(model, testloader):
    loss_list = 0
    n = 0
    model.eval()
    with torch.no_grad():
        for user_id, itm_id, rating in testloader:
            user_id, itm_id, rating = user_id.to(device), itm_id.to(device), rating.to(device)
            outputs = model.forward(user_id, itm_id)
            # compute loss
            loss = F.mse_loss(outputs, rating.float(), reduction="sum")
            loss_list += loss.item()
            n += len(rating)
        return loss_list / n


if not skip_training:
    # YOUR CODE HERE
    model.to(device)
    
    # Define the loss function
    loss_fn = nn.MSELoss()

    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)
    
    # Train the model
    num_epochs = 1000
    
    for epoch in range(num_epochs):
        model.train()

        for i, (user_ids, item_ids, target_rating) in enumerate(trainloader):
            
            #zero the parametr gradients
            optimizer.zero_grad()
            
            # forward pass
            
            predictions = model(user_ids, item_ids)
            
            # compute loss
            loss = loss_fn(predictions, target_rating.float())
            
            # backpropagate
            loss.backward()
            
            # update weights
            optimizer.step()
                
        # Compute the accuracy 
        #accuracy = compute_accuracy(model, testloader)
        test_loss = compute_loss(model, testloader)
            
        #print('Accuracy of the network on the test: %.3f' % accuracy)
        print('Testloss of the network on the test epoch: %d is %.3f' % (epoch, test_loss))
        if test_loss <= 0.88:
            break

    print('Finished Training')
    
    

Testloss of the network on the test epoch: 0 is 0.882
Testloss of the network on the test epoch: 1 is 0.881
Testloss of the network on the test epoch: 2 is 0.879
Finished Training


In [22]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=True)

Do you want to save the model (type yes to confirm)? Yes
Model saved to recsys.pth.


In [23]:
# This cell loads your best model
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)

The next cell tests the accuracy of your best model. It is enough to submit .pth files.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models. Please make sure that your model loads in the cell above.

In [24]:
# This cell tests the accuracy of your best model.

In [25]:
# This cell is reserved for grading

In [26]:
# This cell is reserved for grading

In [27]:
# This cell is reserved for grading

In [28]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading